In [1]:
#!pip install miditok
#!pip install symusic
#!pip install glob
#!pip install torch

In [ ]:
import glob
import random
from typing import List
from collections import defaultdict

import os
import pandas as pd

import numpy as np
from numpy.random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from symusic import Score
from miditok import REMI, TokenizerConfig

from midi2audio import FluidSynth # Import library
from IPython.display import Audio, display

In [ ]:
# Uses 'cuda' if a gpu is detected. Otherwise uses cpu
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Can also set manually
#DEVICE = 'cpu'
#DEVICE = 'cuda'

print(DEVICE)

cuda


In [ ]:
ROOT = "maestro-v3.0.0"            # change if you unpacked elsewhere
meta = pd.read_csv(os.path.join(ROOT, "maestro-v3.0.0.csv"))

def list_midi_files(split):
    paths = meta.loc[meta["split"] == split, "midi_filename"]
    return [os.path.join(ROOT, p) for p in paths]

train_files = list_midi_files("train")        # 962 MIDI files
val_files   = list_midi_files("validation")   # 137
test_files  = list_midi_files("test")         # 177


In [ ]:
type(train_files[0])
train_files[0].encode('utf-8').decode('utf-8')
print(train_files[0].encode('utf-8'))
str.encode(train_files[0], 'utf-8')

b'maestro-v3.0.0\\2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi'


b'maestro-v3.0.0\\2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi'

Tokenizer

In [ ]:
from miditok.pytorch_data import DatasetMIDI, DataCollator

tokenizer = REMI()  # using defaults parameters (constants.py)
train_dataset = DatasetMIDI(
    files_paths=train_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
test_dataset = DatasetMIDI(
    files_paths=test_files,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)
collator = DataCollator(tokenizer.pad_token_id)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collator)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collator)

In [ ]:
len(train_loader), len(test_loader)

(241, 45)

### RNN

In [ ]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(MusicRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        # x: (batch_size, seq_length)
        x = self.embedding(x)  # (batch_size, seq_length, embedding_dim)
        out, hidden = self.rnn(x, hidden)  # out: (batch_size, seq_length, hidden_dim)
        out = self.fc(out)  # (batch_size, seq_length, vocab_size)
        return out, hidden

Training

In [ ]:
def train(model, train_loader, val_loader, vocab_size, num_epochs=20, lr=0.001, device=DEVICE):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # --------- Training ---------
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = batch['input_ids'].to(device)  # (batch_size, seq_length)

            inputs = batch[:, :-1]
            targets = batch[:, 1:]

            optimizer.zero_grad()
            outputs, _ = model(inputs)
            outputs = outputs.reshape(-1, vocab_size)
            targets = targets.reshape(-1)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # --------- Validation ---------
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch['input_ids'].to(device)

                inputs = batch[:, :-1]
                targets = batch[:, 1:]

                outputs, _ = model(inputs)
                outputs = outputs.reshape(-1, vocab_size)
                targets = targets.reshape(-1)

                loss = criterion(outputs, targets)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


# Example usage
if __name__ == "__main__":
    vocab_size = tokenizer.vocab_size
    embedding_dim = 256
    hidden_dim = 512
    num_layers = 2

    model = MusicRNN(vocab_size, embedding_dim, hidden_dim, num_layers)
    train(model, train_loader, test_loader, vocab_size)

Epoch 1/20 | Train Loss: 3.3886 | Val Loss: 2.8900
Epoch 2/20 | Train Loss: 2.7531 | Val Loss: 2.6252
Epoch 3/20 | Train Loss: 2.5753 | Val Loss: 2.5418
Epoch 4/20 | Train Loss: 2.4807 | Val Loss: 2.4895
Epoch 5/20 | Train Loss: 2.3892 | Val Loss: 2.4358
Epoch 6/20 | Train Loss: 2.3119 | Val Loss: 2.4036
Epoch 7/20 | Train Loss: 2.2446 | Val Loss: 2.3865
Epoch 8/20 | Train Loss: 2.1777 | Val Loss: 2.3736
Epoch 9/20 | Train Loss: 2.1124 | Val Loss: 2.3751
Epoch 10/20 | Train Loss: 2.0464 | Val Loss: 2.3829
Epoch 11/20 | Train Loss: 1.9810 | Val Loss: 2.3998
Epoch 12/20 | Train Loss: 1.9148 | Val Loss: 2.4171
Epoch 13/20 | Train Loss: 1.8470 | Val Loss: 2.4387
Epoch 14/20 | Train Loss: 1.7825 | Val Loss: 2.4785
Epoch 15/20 | Train Loss: 1.7208 | Val Loss: 2.5168
Epoch 16/20 | Train Loss: 1.6590 | Val Loss: 2.5640
Epoch 17/20 | Train Loss: 1.5958 | Val Loss: 2.6059
Epoch 18/20 | Train Loss: 1.5385 | Val Loss: 2.6488
Epoch 19/20 | Train Loss: 1.4784 | Val Loss: 2.7093
Epoch 20/20 | Train L

Sampling

In [ ]:
def sample(model, start_token, max_length=100, temperature=1.0, device=DEVICE):
    model = model.to(device)
    model.eval()

    generated = [start_token]
    input_token = torch.tensor([[start_token]], device=device)  # (1, 1)

    hidden = None

    for _ in range(max_length):
        output, hidden = model(input_token, hidden)  # output: (1, 1, vocab_size)
        output = output[:, -1, :]  # take the last output
        output = output / temperature  # adjust randomness

        probs = F.softmax(output, dim=-1)  # (1, vocab_size)
        next_token = torch.multinomial(probs, num_samples=1).item()
        generated.append(next_token)
        if next_token == 2 or next_token == 0: # reach end of sequence
          break

        input_token = torch.tensor([[next_token]], device=device)

    return generated

start_token = tokenizer.special_tokens_ids[1]
generated_sequence = sample(model, start_token, max_length=1024)

print("Generated token sequence:")
print(generated_sequence)

Generated token sequence:
[1, 4, 205, 30, 102, 129, 207, 58, 107, 133, 214, 46, 99, 127, 218, 42, 103, 130, 58, 109, 129, 4, 195, 34, 102, 126, 196, 54, 111, 129, 201, 49, 101, 125, 202, 37, 101, 126, 204, 58, 108, 129, 205, 42, 101, 125, 212, 53, 108, 130, 213, 25, 101, 126, 217, 42, 102, 126, 218, 37, 100, 125, 4, 195, 47, 105, 127, 39, 99, 128, 35, 100, 127, 196, 41, 103, 128, 206, 44, 100, 129, 41, 97, 130, 207, 39, 97, 128, 213, 58, 108, 128, 27, 100, 128, 42, 102, 127, 46, 102, 127, 219, 56, 111, 129, 4, 189, 56, 111, 128, 41, 103, 127, 39, 104, 127, 191, 37, 101, 131, 192, 58, 112, 126, 41, 106, 128, 195, 56, 111, 128, 199, 54, 110, 130, 200, 49, 100, 129, 42, 100, 128, 51, 100, 130, 201, 46, 101, 128, 206, 54, 109, 128, 207, 46, 102, 126, 54, 102, 127, 209, 42, 99, 128, 46, 100, 131, 215, 54, 107, 140, 46, 102, 127, 30, 99, 127, 49, 99, 126, 4, 189, 37, 99, 129, 195, 41, 100, 129, 53, 107, 127, 37, 99, 129, 196, 46, 95, 126, 205, 51, 102, 130, 206, 42, 98, 128, 30, 98, 128, 210

Convert Midi to Wav

In [ ]:
fs = FluidSynth("FluidR3Mono_GM.sf3") # Initialize FluidSynth

output_score = tokenizer.tokens_to_midi([generated_sequence])
output_score.dump_midi(f"rnn.mid")

C:\Users\PC\AppData\Local\Temp\ipykernel_13864\1080284856.py:5: UserWarning: miditok: The `tokens_to_midi` method had been renamed `decode`. It is now depreciated and will be removed in future updates.
  output_score = tokenizer.tokens_to_midi([generated_sequence])


In [ ]:
fs.midi_to_audio("generated_maestro_sample.mid", "rnn.wav")
display(Audio("rnn.wav"))

FileNotFoundError: [WinError 2] The system cannot find the file specified